In [3]:
import os

import mlflow

In [4]:

%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer\\research'

In [5]:
NUMEXPR_MAX_THREADS = 20

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer'

In [8]:
from dataclasses import dataclass
from pathlib import Path


In [9]:
@dataclass(frozen=True)
class CredentialsConfig:
    MLFLOW_TRACKING_URI: str
    MLFLOW_TRACKING_USERNAME: str
    MLFLOW_TRACKING_PASSWORD: str


In [10]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories, save_json

In [11]:
class Credentials:
    def __init__(
        self,
        secrets_filepath = SECRETS_FILE_PATH):
        

        self.secret= read_yaml(secrets_filepath)
    
    def get_mlflow_tracking_credentials(self) -> CredentialsConfig:
        secret = self.secret
        
        model_evaluation_config = CredentialsConfig(
            MLFLOW_TRACKING_URI=self.secret.MLFLOW_TRACKING_URI,
            MLFLOW_TRACKING_USERNAME=self.secret.MLFLOW_TRACKING_USERNAME,
            MLFLOW_TRACKING_PASSWORD = self.secret.MLFLOW_TRACKING_PASSWORD                  
           
        )
        return {
            "MLFLOW_TRACKING_URI": model_evaluation_config.MLFLOW_TRACKING_URI,
            "MLFLOW_TRACKING_USERNAME": model_evaluation_config.MLFLOW_TRACKING_USERNAME,
            "MLFLOW_TRACKING_PASSWORD": model_evaluation_config.MLFLOW_TRACKING_PASSWORD
        }
    

In [12]:
credentials = Credentials()

[2023-08-09 16:13:25,722: INFO: common: yaml file: secrets.yaml loaded successfully]


In [13]:
mlflow_credentials = credentials.get_mlflow_tracking_credentials()

In [14]:
os.environ["MLFLOW_TRACKING_URI"] = mlflow_credentials["MLFLOW_TRACKING_URI"]

In [15]:
os.environ["MLFLOW_TRACKING_USERNAME"] = mlflow_credentials["MLFLOW_TRACKING_USERNAME"]

In [16]:
os.environ["MLFLOW_TRACKING_PASSWORD"] = mlflow_credentials["MLFLOW_TRACKING_PASSWORD"]

In [17]:
PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION="python"

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    params: dict
    tokenizer_path: Path
    metric_file_name: Path
    mlflow_uri: str
    experiment_name: str
    model_path_packed: Path

In [19]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories, save_json


In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        secrets_filepath = SECRETS_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secrets = read_yaml(secrets_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.TrainingArguments
        secrets = self.secrets

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            params = params,            
            metric_file_name = config.metric_file_name,
            mlflow_uri = secrets.MLFLOW_TRACKING_URI,
            experiment_name=config.experiment_name,
            model_path_packed= config.model_path_packed,
            
        )

        return model_evaluation_config

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse
import json
import joblib
import yaml

c:\Users\Admin\anaconda3\envs\mlops\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            #  We decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        

        if mlflow.active_run():
            mlflow.end_run()
        with mlflow.start_run(run_name=self.config.experiment_name) as run:
        
    
            
            mlflow.log_params(self.config.params)
            
            rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    
            rouge_metric = load_metric('rouge')

            score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
                )
            

            # MLflow >= 2.0
            mlflow.doctor() 
            mlflow.pytorch.log_model(model_pegasus, 'model')

            rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
        
            save_json(path=Path(self.config.metric_file_name), data=rouge_dict)        
            
            mlflow.log_metric('rouge1', rouge_dict['rouge1'])
            mlflow.log_metric('rouge2', rouge_dict['rouge2'])
            mlflow.log_metric('rougeL', rouge_dict['rougeL'])
            mlflow.log_metric('rougeLsum', rouge_dict['rougeLsum'])

            run_id = run.info.run_id
            experiment_id = run.info.experiment_id
            model_name = "samsum_pegasus_model"
            subpath = "text_summarization"
            # build the run URI
            run_uri = f'runs:/{run_id}/{subpath}'
            mlflow.end_run()
            print(f'artifact_uri= {mlflow.get_artifact_uri()}')
            print(f'run_uri= {run_uri}')






In [33]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()    
except Exception as e:
    raise e
    

[2023-08-09 23:35:20,983: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-09 23:35:20,987: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-09 23:35:20,989: INFO: common: yaml file: secrets.yaml loaded successfully]
[2023-08-09 23:35:20,990: INFO: common: created directory at: artifacts]
[2023-08-09 23:35:20,991: INFO: common: created directory at: artifacts/model_evaluation]


100%|██████████| 5/5 [02:16<00:00, 27.34s/it]

[2023-08-09 23:37:49,587: INFO: rouge_scorer: Using default tokenizer.]


System information: Windows 10.0.19041
Python version: 3.8.17
MLflow version: 2.5.0
MLflow module location: c:\Users\Admin\anaconda3\envs\mlops\lib\site-packages\mlflow\__init__.py
Tracking URI: https://dagshub.com/Nkifor/mlops-nlp-text-summarizer.mlflow
Registry URI: https://dagshub.com/Nkifor/mlops-nlp-text-summarizer.mlflow
Active experiment ID: 4
Active run ID: b8685dde8d2f4961b95759574f4e985f
Active run artifact URI: mlflow-artifacts:/7c3315d8806244699f2dddb8e4262f96/b8685dde8d2f4961b95759574f4e985f/artifacts
MLflow environment variables: 
  MLFLOW_TRACKING_PASSWORD: fd04345a366444082eb5d904028b0202035a370e
  MLFLOW_TRACKING_URI: https://dagshub.com/Nkifor/mlops-nlp-text-summarizer.mlflow
  MLFLOW_TRACKING_USERNAME: Nkifor
MLflow dependencies: 
  Flask: 2.3.2
  Jinja2: 3.1.2
  alembic: 1.4.1
  click: 8.1.4
  cloudpickle: 2.2.1
  databricks-cli: 0.17.7
  docker: 6.1.3
  entrypoints: 0.4
  gitpython: 3.1.31
  importlib-metadata: 6.8.0
  markdown: 3.4.3
  matplotlib: 3.7.2
  numpy: 1